In [83]:
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import plotly
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import calendar

In [2]:
path = "/Users/peiyu/Desktop/ECE 143/Group Project/gun-violence-data_01-2013_03-2018.csv"
df = pd.read_csv(path)

In [3]:
df.keys()

Index(['incident_id', 'date', 'state', 'city_or_county', 'address', 'n_killed',
       'n_injured', 'incident_url', 'source_url',
       'incident_url_fields_missing', 'congressional_district', 'gun_stolen',
       'gun_type', 'incident_characteristics', 'latitude',
       'location_description', 'longitude', 'n_guns_involved', 'notes',
       'participant_age', 'participant_age_group', 'participant_gender',
       'participant_name', 'participant_relationship', 'participant_status',
       'participant_type', 'sources', 'state_house_district',
       'state_senate_district'],
      dtype='object')

In [4]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['monthday'] = df['date'].dt.day
df['weekday'] = df['date'].dt.weekday
df['loss'] = df['n_killed'] + df['n_injured']

## Time Related Trends of Gun Violence

In [32]:
year_stat = df['year'].value_counts()
year_stat

2017    61401
2016    58763
2015    53579
2014    51854
2018    13802
2013      278
Name: year, dtype: int64

In [98]:
x1 = year_stat.index.tolist()
y1 = year_stat.values.tolist()
trace1 = go.Bar(x = x1, y = y1, name = 'year count', opacity = 0.7,
                marker = dict(
                    color = ['rgba(222,45,38,0.8)', 'rgba(222,45,38,0.8)', 'rgba(222,45,38,0.8)',
                             'rgba(222,45,38,0.8)', 'rgba(204,204,204,1)', 'rgba(204,204,204,1)']))
layout = go.Layout(title = 'Gun Violence Incidents by year')
fig = go.Figure(data = [trace1], layout = layout)
iplot(fig)

In [73]:
month_count = defaultdict(int)
for i in range(len(df)):
    if df['year'][i] in [2014, 2015, 2016, 2017]:
        month_count[df['month'][i]] += 1
month_mean = defaultdict(int)
for i in month_count.keys():
    month_mean[i] = month_count[i] / len([2014, 2015, 2016, 2017])

In [111]:
x2 = list(month_mean.keys())
y2 = list(month_mean.values())
trace2 = go.Bar(x = [calendar.month_abbr[int(x)] for x in x2], y = y2, name = 'month', opacity = 0.7, marker = dict(color = 'rgba(200,200,3,0.5)'))
layout = go.Layout(title = 'Average number of Gun Violence Incidents by Months')
fig = go.Figure(data = [trace2], layout = layout)
iplot(fig)